MLflow setup:

- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem
The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [3]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [4]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-2")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/06/14 01:39:32 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-2' does not exist. Creating a new experiment.


default artifacts URI: 'mlflow-artifacts:/3/ce2c43c2fa9e4ab0a20ba3937c7409d1/artifacts'


In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [13]:
# Retrieve the list of registered models
registered_models = client.search_registered_models()

# Print the names of the registered models
for model in registered_models:
    print(model.name)

iris-classifier
nyc-taxi-regressor


In [17]:
# List runs in the specified experiment
runs = client.search_runs(experiment_ids=['3'])

# Get the run ID of the first run
run_id = runs[0].info.run_id

# Register the model
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/06/14 01:51:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1718329882054, current_stage='None', description='', last_updated_timestamp=1718329882054, name='iris-classifier', run_id='ce2c43c2fa9e4ab0a20ba3937c7409d1', run_link='', source='mlflow-artifacts:/3/ce2c43c2fa9e4ab0a20ba3937c7409d1/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [18]:
# List runs in the specified experiment
runs = client.search_runs(experiment_ids=['3'])
runs

[<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "ce2c43c2fa9e4ab0a20ba3937c7409d1", '
                              '"artifact_path": "models", "utc_time_created": '
                              '"2024-06-14 01:39:32.860191", "flavors": '
                              '{"python_function": {"model_path": "model.pkl", '
                              '"predict_fn": "predict", "loader_module": '
                              '"mlflow.sklearn", "python_version": "3.9.19", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "sklearn": '
                              '{"pickled_model": "model.pkl", '
                              '"sklearn_version": "1.5.0", '
                              '"serialization_format": "cloudpickle", "code": '
                              'null}}, "model_uuid": '
                              

In [19]:
# Get the run ID of the first run
run_id = runs[0].info.run_id
run_id

'ce2c43c2fa9e4ab0a20ba3937c7409d1'